# Rolling 4 weeks Retention Rate
### of Resellers in a dropshipping company

In a dropshipping company, a resellers who makes an order in e.g. week n, is considered as retaind for week n+1, week n+2, and week n+1. In this report, for each week, I count the number of active users in the interval [week n-3, week n].

with base AS (SELECT
    `orders`.`ID` AS `OrderID`,
    `orders`.`UserID` AS `UserID`,
    `orders`.`CreatedAt` AS `OrderDate`,
    `orders`.`FakeDelete` AS `FakeDelete`,
    `Users`.`TotalOrders` AS `TotalOrders`,
    `Users`.`CreatedAt` AS `JoinDate`,
    `Vw Orders Is Net`.`isGross` AS `isGross`,
    `Vw Users First Order`.`FirstOrderAt` AS `ActivationDate`,
    DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`) + 1 AS `Day`,
    CEILING((DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`)+1)/7) as `Week`,
    DATEDIFF( `orders`.`CreatedAt`, coalesce( LAG(`orders`.`CreatedAt`) OVER ( PARTITION BY UserID ORDER BY `orders`.`CreatedAt`), 0)
      ) AS DistanceFromPreviousOrder,
    ROW_NUMBER() OVER (PARTITION BY UserID ORDER BY orders.CreatedAt DESC ) AS OrderOrder,
    -- DATE_ADD(Day, INTERVAL 29 DAY) AS `RetentionDue`
    DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`) + 30 AS `RetentionDue`,
    DATEDIFF(`orders`.`CreatedAt`, `Vw Users First Order`.`FirstOrderAt`) - 29 AS `Start`
FROM
    `orders`
    INNER JOIN `users` AS `Users` ON `orders`.`UserID` = `Users`.`ID`
    INNER JOIN `vw_orders_is_net` AS `Vw Orders Is Net` ON `orders`.`ID` = `Vw Orders Is Net`.`OrderID`
    INNER JOIN `vw_users_first_order` AS `Vw Users First Order` ON `Users`.`ID` = `Vw Users First Order`.`UserID`
WHERE
   (`Vw Orders Is Net`.`isGross` = 1)
   AND (`orders`.`FakeDelete` = 0)
ORDER BY `orders`.`UserID`),

myOrder AS (SELECT Week, UserID FROM base GROUP BY Week, UserID ORDER BY Week),

myWeeks AS (SELECT DISTINCT Week FROM myOrder ORDER BY Week),

myJoin AS (SELECT myWeeks.Week AS WWeek, myOrder.Week as OWeek, UserID
FROM myWeeks, myOrder
WHERE myOrder.Week BETWEEN (myWeeks.Week-3) AND myWeeks.Week),

myFinal AS (SELECT WWeek AS Week, COUNT(DISTINCT OWeek) AS Duration, MIN(OWeek) AS StartWeek, MAX(OWeek) AS EndWeek, COUNT(DISTINCT UserID) AS ActivesInInterval
FROM  myJoin
GROUP BY Week),

allActivated AS (SELECT MAX(ActivesInInterval) AS AllActivation FROM myFinal)

SELECT Week, StartWeek, Duration, EndWeek, ActivesInInterval, ActivesInInterval/AllActivation AS PercentActivesInInterval
FROM  myFinal, allActivated
WHERE Week > 4;